<a href="https://colab.research.google.com/github/wiederm/notebooks/blob/main/UmbrellaPMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# check that everything works
import condacolab
condacolab.check()

In [ ]:
# install packages
!mamba install -c conda-forge matplotlib pymbar openmm plotly > /dev/null
# cloning data repo
!git clone https://github.com/wiederm/notebooks.git

In [ ]:
# importing all packages
import numpy as np
import pymbar
import glob
from openmm import unit

In [ ]:
# loading data and initialize variables
# get number of umbrealla windows
data_path = 'notebooks/data_for_pmf'
K = len(glob.glob(f'{data_path}/traj*.npy'))
number_of_frames = 500
print(f'Nr of windows: {K}') # number of umbrellas

# initialize N_k, in which N_k[k] which is the number of snapshots from umbrella simulation k
N_k = np.zeros([K], dtype=np.int32)
# initialize restrain_k, in which restraint_k[k] is the restraint center location for umbrella k
restraint_k = np.zeros([K, 3])
# initialize com_for_all_frames, in which com_for_all_frames[k] are all center of masses from umbrella simulation k
com_for_all_frames = np.zeros((K, number_of_frames,3), dtype=np.float64)
# number of total samples from all umbrella simulations
N_tot_sampes=0
# initialize bin_kn, for each umbrella simulation k each frame is assigned a bin
bin_kn = np.zeros([K,number_of_frames], np.int32)


# Read in umbrella and centers and fill restraint_k
with open(f'{data_path}/coordinates.dat', 'r') as f:
    lines = f.readlines()
    for k in range(K):
        # Parse line k.
        line = lines[k+1]
        tokens = line.split(',')
        restraint_k[k] = float(tokens[1]), float(tokens[2]), float(tokens[3]) # spring center locatiomn (in nm)

# initialize u_kln which is the energy of snapshot n from umbrella simulation k evaluated at umbrella l
u_kln = np.zeros([K,K,number_of_frames], np.float64) 

In [ ]:
# load the masses for the system
mass_list = np.load(open(f'notebooks/data_for_pmf/mass.npy', 'rb'))
scaled_masses = mass_list / mass_list.sum()
# fill com_for_all_frames, N_k, and N_tot_sampes
print('Calculate COM ...')
for k in range(K):
    trajectory = np.load(open(f'notebooks/data_for_pmf/traj_{k}.npy', 'rb'))
    for frame in range(len(trajectory)):
        # extract ligand coordinates
        ligand_coordinates = trajectory[frame]
        # calculate COM
        com_for_all_frames[k,frame] = np.matmul(ligand_coordinates.T, scaled_masses)
    N_k[k] = number_of_frames
    N_tot_sampes += number_of_frames


In [ ]:
# plot the umbrella centers
import plotly.express as px
fig = px.scatter_3d(x=list(restraint_k[:, [0]].flatten()),y=list(restraint_k[:, [1]].flatten()), z=list(restraint_k[:, [2]].flatten()), size=[0.5]*len(restraint_k))
              #color='species')
fig.show()


In [ ]:
# generate bins. adding additional bin betwen all umbrella windows
print('Generating bins ...')
bin_center = []
N_bins = 0
# for each two umbrella location, add one in between
for i in range(0,len(restraint_k)-1):
    bin_center.append(restraint_k[i])
    bin_center.append(restraint_k[i] + (restraint_k[i+1] - restraint_k[i])/2 )
    N_bins += 1
bin_center.append(restraint_k[-1])
bin_center=np.array(bin_center)
print(f'total number of bins: {N_bins}')

In [ ]:
# plot the bin centers
color=['red', 'blue']*(K-1)
print(len(color))
fig = px.scatter_3d(x=list(bin_center[:, [0]].flatten()),y=list(bin_center[:, [1]].flatten()), z=list(bin_center[:, [2]].flatten()), size=[0.5]*len(bin_center), color=color + ['red'])
              #color='species')
fig.show()

In [ ]:
# now bin the data to each bin
print("Binning data...")
# Bin data

for k in range(K): # for each window
    # initialize distance_m, which contains the distance from each frame to each bin center 
    distance_m = np.zeros([N_bins,number_of_frames], np.float32)
    for bin in range(N_bins): # iterate through bins and  calculate the distance for each frame from simulation K to each bin center
        distance_m[bin] = np.linalg.norm(com_for_all_frames[k]- bin_center[bin],  axis=1)
    # obtain the index for each minimum value along each column (representing the distance for a single fraem to each bin) 
    bin_kn[k] = distance_m.argmin(axis=0)


In [ ]:
print("Evaluating reduced potential energies...")
temperature = 303.15 * unit.kelvin
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
kT = kB * temperature
f_k = 1 * unit.kilocalories_per_mole / unit.angstrom**2

# for each windows
for k in range(K):
    # for each window
    for l in range(K):   
        # Compute energy of snapshot n from simulation k in umbrella potential l
        u_kln[k,l] = (((np.linalg.norm(com_for_all_frames[k] - restraint_k[l])) * unit.angstrom **2 ) * f_k) / kT  
print('Finished.')

In [ ]:
import pymbar
# Initialize MBAR.
print("Running MBAR...")
mbar = pymbar.MBAR(u_kln, N_k, verbose = True, initialize='BAR')
u_kn = np.zeros([K, number_of_frames]) # u_kn[k,n] is the reduced potential energy without umbrella restraints of snapshot n of umbrella simulation k
# Compute PMF in unbiased potential (in units of kT).
results = mbar.computePMF(u_kn, bin_kn, N_bins, return_dict=True)

In [ ]:
# Write out PMF
f_i = results['f_i']
df_i = results['df_i']
print("PMF (in units of kT)")
print("%8s %8s %8s" % ('bin', 'f', 'df'))
for i in range(N_bins):
    print("%8.1f %8.3f %8.3f" % (i, f_i[i], df_i[i]))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(f_i)
plt.xlabel('distance [A]')
plt.ylabel('Relativ free energy [kT]')
plt.plot()
